#Import Libraries

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
import plotly.express as px
import itertools
import functools
import operator

#Read Data

In [3]:
df_promo = pd.read_csv('raw_data/promos.csv')

In [4]:
df_promo

,Campaign,Priority,HN,Notes,Target-Audience(Male/Female/Kids),Promo Duration Seconds
0,1,M,DIG-08357,MBC1,Female/Male,102
1,2,H,DIG-08358,MBC1,Female,80
2,3,H,DIG-08368,MBC1,Female/Male,60
3,4,H,DIG-08374,MBC1,Female/Male,55
4,5,H,DIG-08376,MBC1,Female/Male,35
5,6,H,DIG-08378,MBC1,Female/Male,35
6,7,L,DIG-08362,MBC1,Female,40


In [5]:
df_scheduel = pd.read_csv('raw_data/assignment_dataset.csv')

In [6]:
df_scheduel

,timestamp,end_timestamp,Slot,Segment No.,Total Airtime (seconds),target_audience,reach_avg
0,30/01/2024 6:00,30/01/2024 6:16,1,1,240,female,584.875000
1,30/01/2024 6:16,30/01/2024 6:25,1,2,240,female,610.777778
2,30/01/2024 6:25,30/01/2024 6:34,1,3,240,female,614.666667
3,30/01/2024 7:00,30/01/2024 7:15,2,1,360,female,617.200000
4,30/01/2024 7:15,30/01/2024 7:21,2,2,360,female,684.833333
...,...,...,...,...,...,...,...
61,31/01/2024 4:30,31/01/2024 4:39,25,1,120,female/male,701.777778
62,31/01/2024 4:39,31/01/2024 4:48,25,2,120,female/male,685.111111
63,31/01/2024 5:00,31/01/2024 5:14,26,1,240,female,667.857143
64,31/01/2024 5:14,31/01/2024 5:22,26,2,240,female,676.750000


#Data Cleaning

In [7]:
df_scheduel['Slot'] = df_scheduel['Slot'].astype(str)

In [8]:
df_scheduel['timestamp'] = [datetime.strptime(x,'%d/%m/%Y %H:%M') for x in df_scheduel['timestamp']]
df_scheduel['end_timestamp'] = [datetime.strptime(x,'%d/%m/%Y %H:%M') for x in df_scheduel['end_timestamp']]

df_scheduel['length'] = df_scheduel['end_timestamp'] - df_scheduel['timestamp']

In [9]:
df_promo['Target-Audience(Male/Female/Kids)'] = [x.lower() for x in df_promo['Target-Audience(Male/Female/Kids)']]

#Average Reach Analysis

##Graphical Represenation

In [10]:
df = px.data.tips()
fig = px.box(df_scheduel, y="reach_avg",)
fig.update_traces(quartilemethod="linear") # or "inclusive", or "linear" by default
fig.show()

##Priority Proxy from Average Reach

In [11]:
Qs = df_scheduel['reach_avg'].quantile([0, .25, .5, .75, 1])
condition = [
    ((df_scheduel['reach_avg']>=Qs[0]) & (df_scheduel['reach_avg']<Qs[0.5])),
     ((df_scheduel['reach_avg']>=Qs[0.5]) & (df_scheduel['reach_avg']<Qs[0.75])),
      df_scheduel['reach_avg']>Qs[0.75]]

Values = ['L','M','H']

df_scheduel['Priority_scheduel'] = np.select(condition,Values)

#Main Code

In [12]:
df_scheduel

,timestamp,end_timestamp,Slot,Segment No.,Total Airtime (seconds),target_audience,reach_avg,length,Priority_scheduel
0,2024-01-30 06:00:00,2024-01-30 06:16:00,1,1,240,female,584.875000,0 days 00:16:00,L
1,2024-01-30 06:16:00,2024-01-30 06:25:00,1,2,240,female,610.777778,0 days 00:09:00,L
2,2024-01-30 06:25:00,2024-01-30 06:34:00,1,3,240,female,614.666667,0 days 00:09:00,L
3,2024-01-30 07:00:00,2024-01-30 07:15:00,2,1,360,female,617.200000,0 days 00:15:00,L
4,2024-01-30 07:15:00,2024-01-30 07:21:00,2,2,360,female,684.833333,0 days 00:06:00,L
...,...,...,...,...,...,...,...,...,...
61,2024-01-31 04:30:00,2024-01-31 04:39:00,25,1,120,female/male,701.777778,0 days 00:09:00,L
62,2024-01-31 04:39:00,2024-01-31 04:48:00,25,2,120,female/male,685.111111,0 days 00:09:00,L
63,2024-01-31 05:00:00,2024-01-31 05:14:00,26,1,240,female,667.857143,0 days 00:14:00,L
64,2024-01-31 05:14:00,2024-01-31 05:22:00,26,2,240,female,676.750000,0 days 00:08:00,L


In [13]:
final_dfs_list = []
slot_list = []
for slot in df_scheduel['Slot'].unique():

  df_slot = df_scheduel[df_scheduel['Slot']==slot]
  duration = df_promo['Promo Duration Seconds']
  campaign = df_promo['Campaign']
  target_group = df_promo['Target-Audience(Male/Female/Kids)']
  priority = df_promo['Priority']

  max_duration = df_slot['Total Airtime (seconds)'].unique()[0]
  min_ads = df_slot.shape[0]-1
  target_audience = df_slot['target_audience'].unique()[0]
  timing_priority = df_slot['Priority_scheduel'].unique()


  #combination of target audience that conforms to the hard constrains
  audience_list = [aud for i in range(len(duration), 0, -1)
            for seq,camp,aud,prio in zip(itertools.combinations(duration, i),
                                        itertools.combinations(campaign, i),
                                        itertools.combinations(target_group, i),
                                        itertools.combinations(priority, i))
            if sum(seq) < max_duration #promo duration less than airtime duration
            and target_audience in aud #target audience of a promo is matching target audience in a segment
            ]
  #logic for non female/male promo occuring in female/male
  if target_audience=='female/male':
    audience_list_flag = [True if len(set(tuple))==1 and list(set(tuple))[0]=='female/male' else False for tuple in audience_list]
    audience_list_filtered = [x for x,y in zip(audience_list,audience_list_flag) if y==True]

  else:
    audience_list_flag = [True for x in audience_list]
    audience_list_filtered = audience_list.copy()

  campaign_list = [camp for i in range(len(duration), 0, -1)
            for seq,camp,aud in zip(itertools.combinations(duration, i),
                                itertools.combinations(campaign, i),
                                itertools.combinations(target_group, i)
                                )
            if sum(seq) <= max_duration #promo duration less than airtime duration
            and target_audience in aud #target audience of a promo is matching target audience in a segment
            ]

  #get the list of campaign combination based on audience logic
  campaign_list_filtered = [x for x,y in zip(campaign_list,audience_list_flag) if y==True]

  #create logic for priority so that L<M<H
  priority_list = [prio for i in range(len(duration), 0, -1)
          for seq,camp,aud,prio in zip(itertools.combinations(duration, i),
                                      itertools.combinations(campaign, i),
                                      itertools.combinations(target_group, i),
                                      itertools.combinations(priority, i))
          if sum(seq) < max_duration #promo duration less than airtime duration
          and target_audience in aud #target audience of a promo is matching target audience in a segment
          ]

  priority_list_filtered = [x for x,y in zip(priority_list,audience_list_flag) if y==True]
  priority_list_flag = [True if len(set(timing_priority).intersection(set(d))) == len(timing_priority) else False for d in priority_list_filtered]

  if len([x for x,y in zip(campaign_list_filtered,priority_list_flag) if y==True]) == 0:
    campaign_list_filtered_ = campaign_list_filtered.copy()
  else:
    campaign_list_filtered_ = [x for x,y in zip(campaign_list_filtered,priority_list_flag) if y==True]

  df_list = []
  for campaign_combination in campaign_list_filtered_:
    campaign_combination_list = []
    for campaign in campaign_combination:
      df_ = df_promo[df_promo['Campaign']==campaign]
      campaign_combination_list.append(df_)
    df_list.append(pd.concat(campaign_combination_list,axis=0))

  final_dfs_list.append(df_list[0])
  slot_list.append([str(slot)]*len(df_list[0]))

In [14]:
df_final = pd.concat(final_dfs_list,axis=0)
slot_column = functools.reduce(operator.iconcat, slot_list, [])

df_final['Slot'] = slot_column
df_final['Segment No.'] = df_final.groupby('Slot').cumcount()+1

In [15]:
df_final

,Campaign,Priority,HN,Notes,Target-Audience(Male/Female/Kids),Promo Duration Seconds,Slot,Segment No.
2,3,H,DIG-08368,MBC1,female/male,60,1,1
3,4,H,DIG-08374,MBC1,female/male,55,1,2
4,5,H,DIG-08376,MBC1,female/male,35,1,3
5,6,H,DIG-08378,MBC1,female/male,35,1,4
6,7,L,DIG-08362,MBC1,female,40,1,5
...,...,...,...,...,...,...,...,...
2,3,H,DIG-08368,MBC1,female/male,60,26,1
3,4,H,DIG-08374,MBC1,female/male,55,26,2
4,5,H,DIG-08376,MBC1,female/male,35,26,3
5,6,H,DIG-08378,MBC1,female/male,35,26,4


In [16]:
df_merged = pd.merge(df_scheduel,df_final,on=['Slot','Segment No.'],how='left')

In [17]:
df_merged = df_merged[df_merged['Campaign'].notna()].reset_index(drop=True)
df_final['Cumulative_duratoin'] = df_final.groupby('Slot')['Promo Duration Seconds'].cumsum()
df_merged['Promo Duration'] = pd.to_timedelta(df_merged['Promo Duration Seconds'], unit='s')

In [18]:
df_merged

,timestamp,end_timestamp,Slot,Segment No.,Total Airtime (seconds),target_audience,reach_avg,length,Priority_scheduel,Campaign,Priority,HN,Notes,Target-Audience(Male/Female/Kids),Promo Duration Seconds,Promo Duration
0,2024-01-30 06:00:00,2024-01-30 06:16:00,1,1,240,female,584.875000,0 days 00:16:00,L,3.0,H,DIG-08368,MBC1,female/male,60.0,0 days 00:01:00
1,2024-01-30 06:16:00,2024-01-30 06:25:00,1,2,240,female,610.777778,0 days 00:09:00,L,4.0,H,DIG-08374,MBC1,female/male,55.0,0 days 00:00:55
2,2024-01-30 06:25:00,2024-01-30 06:34:00,1,3,240,female,614.666667,0 days 00:09:00,L,5.0,H,DIG-08376,MBC1,female/male,35.0,0 days 00:00:35
3,2024-01-30 07:00:00,2024-01-30 07:15:00,2,1,360,female,617.200000,0 days 00:15:00,L,1.0,M,DIG-08357,MBC1,female/male,102.0,0 days 00:01:42
4,2024-01-30 07:15:00,2024-01-30 07:21:00,2,2,360,female,684.833333,0 days 00:06:00,L,2.0,H,DIG-08358,MBC1,female,80.0,0 days 00:01:20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,2024-01-31 04:30:00,2024-01-31 04:39:00,25,1,120,female/male,701.777778,0 days 00:09:00,L,3.0,H,DIG-08368,MBC1,female/male,60.0,0 days 00:01:00
59,2024-01-31 04:39:00,2024-01-31 04:48:00,25,2,120,female/male,685.111111,0 days 00:09:00,L,4.0,H,DIG-08374,MBC1,female/male,55.0,0 days 00:00:55
60,2024-01-31 05:00:00,2024-01-31 05:14:00,26,1,240,female,667.857143,0 days 00:14:00,L,3.0,H,DIG-08368,MBC1,female/male,60.0,0 days 00:01:00
61,2024-01-31 05:14:00,2024-01-31 05:22:00,26,2,240,female,676.750000,0 days 00:08:00,L,4.0,H,DIG-08374,MBC1,female/male,55.0,0 days 00:00:55


#New Optimized Timestamps

In [19]:
schd_start_lst= []
schd_end_lst= []
promo_start_lst= []
promo_end_lst= []
for schd_start,schd_end,schd_len,promo_len,slot,i in zip(df_merged['timestamp'],df_merged['end_timestamp'],df_merged['length'],df_merged['Promo Duration'],df_merged['Slot'],list(range(0,df_merged.shape[0]))):
  if i==0:
    schd_start_lst.append(schd_start)
    schd_end_lst.append(schd_end)

    promo_start_ts = schd_end
    promo_end_ts = promo_start_ts + promo_len
    promo_start_lst.append(promo_start_ts)
    promo_end_lst.append(promo_end_ts)

  elif int(slot)-int(df_merged['Slot'][i-1])>1:
    schd_start_lst.append(schd_start)
    schd_end_lst.append(schd_end)

    promo_start_ts = schd_end
    promo_end_ts = promo_start_ts + promo_len
    promo_start_lst.append(promo_start_ts)
    promo_end_lst.append(promo_end_ts)


  else:
    schd_start_ts = promo_end_lst[i-1]
    schd_end_ts = schd_start_ts + schd_len

    promo_start_ts = schd_end_ts
    promo_end_ts = promo_start_ts + promo_len

    schd_start_lst.append(schd_start_ts)
    schd_end_lst.append(schd_end_ts)

    promo_start_lst.append(promo_start_ts)
    promo_end_lst.append(promo_end_ts)



In [20]:
df_merged['schd_start_ts']=schd_start_lst
df_merged['schd_end_ts']=schd_end_lst
df_merged['promo_start_ts']=promo_start_lst
df_merged['promo_end_ts']=promo_end_lst

In [21]:
df_merged

,timestamp,end_timestamp,Slot,Segment No.,Total Airtime (seconds),target_audience,reach_avg,length,Priority_scheduel,Campaign,Priority,HN,Notes,Target-Audience(Male/Female/Kids),Promo Duration Seconds,Promo Duration,schd_start_ts,schd_end_ts,promo_start_ts,promo_end_ts
0,2024-01-30 06:00:00,2024-01-30 06:16:00,1,1,240,female,584.875000,0 days 00:16:00,L,3.0,H,DIG-08368,MBC1,female/male,60.0,0 days 00:01:00,2024-01-30 06:00:00,2024-01-30 06:16:00,2024-01-30 06:16:00,2024-01-30 06:17:00
1,2024-01-30 06:16:00,2024-01-30 06:25:00,1,2,240,female,610.777778,0 days 00:09:00,L,4.0,H,DIG-08374,MBC1,female/male,55.0,0 days 00:00:55,2024-01-30 06:17:00,2024-01-30 06:26:00,2024-01-30 06:26:00,2024-01-30 06:26:55
2,2024-01-30 06:25:00,2024-01-30 06:34:00,1,3,240,female,614.666667,0 days 00:09:00,L,5.0,H,DIG-08376,MBC1,female/male,35.0,0 days 00:00:35,2024-01-30 06:26:55,2024-01-30 06:35:55,2024-01-30 06:35:55,2024-01-30 06:36:30
3,2024-01-30 07:00:00,2024-01-30 07:15:00,2,1,360,female,617.200000,0 days 00:15:00,L,1.0,M,DIG-08357,MBC1,female/male,102.0,0 days 00:01:42,2024-01-30 06:36:30,2024-01-30 06:51:30,2024-01-30 06:51:30,2024-01-30 06:53:12
4,2024-01-30 07:15:00,2024-01-30 07:21:00,2,2,360,female,684.833333,0 days 00:06:00,L,2.0,H,DIG-08358,MBC1,female,80.0,0 days 00:01:20,2024-01-30 06:53:12,2024-01-30 06:59:12,2024-01-30 06:59:12,2024-01-30 07:00:32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,2024-01-31 04:30:00,2024-01-31 04:39:00,25,1,120,female/male,701.777778,0 days 00:09:00,L,3.0,H,DIG-08368,MBC1,female/male,60.0,0 days 00:01:00,2024-01-31 02:39:35,2024-01-31 02:48:35,2024-01-31 02:48:35,2024-01-31 02:49:35
59,2024-01-31 04:39:00,2024-01-31 04:48:00,25,2,120,female/male,685.111111,0 days 00:09:00,L,4.0,H,DIG-08374,MBC1,female/male,55.0,0 days 00:00:55,2024-01-31 02:49:35,2024-01-31 02:58:35,2024-01-31 02:58:35,2024-01-31 02:59:30
60,2024-01-31 05:00:00,2024-01-31 05:14:00,26,1,240,female,667.857143,0 days 00:14:00,L,3.0,H,DIG-08368,MBC1,female/male,60.0,0 days 00:01:00,2024-01-31 02:59:30,2024-01-31 03:13:30,2024-01-31 03:13:30,2024-01-31 03:14:30
61,2024-01-31 05:14:00,2024-01-31 05:22:00,26,2,240,female,676.750000,0 days 00:08:00,L,4.0,H,DIG-08374,MBC1,female/male,55.0,0 days 00:00:55,2024-01-31 03:14:30,2024-01-31 03:22:30,2024-01-31 03:22:30,2024-01-31 03:23:25


In [24]:
df_merged.groupby(by='Priority')['reach_avg'].sum()

Priority
H    64716.861523
L     5747.066667
M    19765.614256
Name: reach_avg, dtype: float64

In [23]:
df_merged.to_csv('deliverables/optimized_promo.csv')